# RAG 절차
```
[ RAG 구현 절차 ]
1.	문서의 내용을 읽는다(document_loader를 이용)
(1)	https://python.langchain.com/v0.2/docs/integrations/document_loaders/ 
(2)	https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/
%pip install --upgrade --quiet  docx2txt
2.	문서를 쪼갠다(한번에 이해하고 처리할 수 있는 입력+출력 토큰수가 제한)
(1)	 https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries 
%pip install -qU langchain-text-splitters
3.	쪼갠 문서를 임베딩하여 vector database에 넣음
(1)	OpenAIEmbeddings나 UpstageEmbeddings이용해서 임베딩
(2)	https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/  
%pip install -q langchain-chroma
4.	질문을 이용해 유사도 검색
5.	유사도 검색한 문서를 LLM에 질문으로 전달하여 답변 얻음(제공되는 Prompt활용)
(1)	https://python.langchain.com/v0.2/docs/tutorials/rag/
%pip install –q langchain langchainhub
http://smith.langchain.com에서 key생성 .env key 추가
https://smith.langchain.com/o/5af7263c-03ac-4284-bd0d-898f42dc4eb7/settings/apikeys
```

# 0. 패키지 설치

In [1]:
# 문서 읽어오기
%pip install --upgrade --quiet  docx2txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 텍스트를 청크로 나누는 기능만 있는 경량 모듈
%pip install -qU langchain-text-splitters

In [4]:
# 벡터DB (로컬DB)
%pip install -q langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [5]:
# 제공되는 prompt 사용
%pip install -q langchain langchainhub

Note: you may need to restart the kernel to use updated packages.


# 1. 문서 읽기(X)

In [17]:
%%time
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
document = loader.load()

CPU times: total: 3.7 s
Wall time: 3.69 s


In [18]:
len(document)

1

In [19]:
document[0].page_content[:200]

'소득세법\n\n소득세법\n\n[시행 2025. 7. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]\n\n기획재정부(재산세제과(양도소득세)) 044-215-4312\n\n기획재정부(소득세제과(근로소득)) 044-215-4216\n\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233\n\n기획재정부(소득세제과(사업소득, 기타소득)) 044-2'

# 2. 문서를 쪼개면서 읽기(o)

In [20]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 문서를 쪼갤때 1500글자씩 쪼개
    chunk_overlap=200
)
# 1번째 chunk 1~1450글자
# 2번째 chunk 1250~1750글자
document = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() - start
print('문서 쪼개면서 읽는 시간 :', runtime)

문서 쪼개면서 읽는 시간 : 3.9636147022247314


In [21]:
# chunk 갯수
len(document)

183

In [22]:
len(document[0].page_content)

1464

In [23]:
# chunk의 글자수
# [ len(doc.page_content) for doc in document ]
print(max(len(doc.page_content) for doc in document))
print(min(len(doc.page_content) for doc in document))

1497
1055


# 3. 쪼갠 문서를 임베딩 -> 벡터 데이터베이스 저장
- 임베딩 모델 : upstage의 solar-embedding-1-large
- 벡터 데이터베이스 : chroma

In [25]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
# https://python.langchain.com/v0.2/docs/integrations/text_embedding/upstage/#usage

load_dotenv()
embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large"
#     model="embedding-query"
)

In [26]:
doc_result = embeddings.embed_documents(
    ["소득세법 관련 내용입니다.", document[0].page_content]
)
# print(doc_result)
print(len(doc_result), len(doc_result[0]))

2 4096


In [30]:
len(embeddings.embed_query("소득세"))

4096

In [31]:
%%time
from langchain_chroma import Chroma
# 데이터를 처음 저장할 때
database = Chroma.from_documents(
    documents=document,
    embedding=embeddings,
    collection_name="tax-collection", # 생략시 이름 랜덤
    persist_directory="./chroma"      # 생략시 로컬데이터베이스에 저장 안됨. 프로그램 종료시 DB 날라감
)
# 이미 저장된 vector DB를 사용할 때
# database = Chroma(
#     embedding_function=embedding,
#     collection_name="tax-collection",
#     persist_directory='./chroma'
# )

InvalidArgumentError: Collection expecting embedding with dimension of 3072, got 4096

# 4. vector DB에 질문과 유사도 검색(답변 생성을 위한 retrieval)

In [53]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, 
                                           k=3) # 기본 k는 4

In [54]:
retrieved_docs

[Document(id='256637b5-8be5-47cb-ae39-313b21b7ac3c', metadata={'source': './tax_docs/소득세법(법률)(제20615호)(20250701).docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(

# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM 전달하여 답변 생성

In [55]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [56]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
[context]는 다음과 같아요
{retrieved_docs}
Question : {query}"""

In [59]:
ai_message = llm.invoke(prompt)

In [61]:
print(ai_message.content)

연봉 5천만원(50,000,000원)인 직장인의 소득세액을 계산하기 위해서는 먼저 과세표준과 세율 구간을 살펴볼 필요가 있습니다.

한국의 개인 소득세는 누진세율 구조로 적용되며, 2024년 기준 세율 구간은 다음과 같습니다:

| 과세표준 구간(원)             | 세율(%) | 세액감면(원) |
|------------------------|------------|--------------|
| 1,200만원 이하           | 6%         | 0            |
| 1,200만원 초과 ~ 4,600만원 이하 | 15%        | 108만원       |
| 4,600만원 초과 ~ 8,800만원 이하 | 24%        | 522만원       |
| 8,800만원 초과             | 35%        | 1,490만원     |

※ 다만, 일반적으로는 공제금액(근로소득공제, 인적공제 등)을 차감한 과세표준에 세율을 적용하는 방식입니다. 이번 예에서는 근로소득공제만 고려하겠습니다.

**1. 근로소득공제 계산**

2024년 기준 근로소득공제는 다음과 같습니다:

- 연봉 5,000만원인 경우, 근로소득공제는 아래 표에 따라 계산됩니다.

| 연봉(원)                         | 근로소득공제액(원)                                  |
|------------------------|---------------------------------------------------|
| 5,000만원 이하            | 연봉의 55% + 1,000,000원 또는 최소 공제액 도 적용됨 |

하지만, 현재 법령상 계산 시 예를 들면:

- 공제액 ≈ 연봉 × 55% + 1,000,000원

구체적인 계산:

- 5,000만원 × 55% = 2,750만원
- plus, 1,000,000원 (추가 공제액, 하지만 연봉이 5천만원인 경우 연봉공제 한도 내에서 공제되므

# 5. Augmentation 을 위한 제공되는 Prompt 활용하여 langchain으로 답변 생성

In [ ]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

In [62]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

### RetrievalQA를 통해 LLM 전달 (create_retrieval_chain이 대체)
```
query -> retriever 전달(벡터 검색 수행)
-> retrieval문서 -> prompt의 {context}에 삽입
-> query -> prompt의 {question}에 삽입
```

In [64]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(search_kwargs={'k':5}),
    chain_type_kwargs={"prompt":prompt}
)

In [65]:
ai_message = qa_chain.invoke({"query":query})

In [66]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 구체적인 세율과 공제액에 따라 달라지기 때문에 명확한 금액을 제시하기 어렵습니다. 일반적으로 과세표준에 따른 누진세율이 적용되며, 공제액이나 세액공제 여부에 따라 차이가 납니다. 따라서 정확한 세액을 계산하려면 세금 계산기 또는 세무전문가의 상담이 필요합니다.'}